In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        
        print(os.path.join(dirname, filename))
import warnings

def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

# Any results you write to the current directory are saved as output.

/kaggle/input/videogame/train.csv
/kaggle/input/testvideogame/test.csv


In [2]:
import numpy as np
import pandas as pd 
import calendar
import seaborn as sns
import time
from scipy import stats
from scipy.stats import norm, skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder,RobustScaler,StandardScaler,PolynomialFeatures,Imputer,MinMaxScaler
from sklearn.linear_model import ElasticNet,Lasso,BayesianRidge,LassoLarsIC,LinearRegression,RidgeCV,LassoCV,ElasticNetCV
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator,TransformerMixin, RegressorMixin,clone
from sklearn.model_selection import KFold,cross_val_score,train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score,make_scorer
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm 
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
%matplotlib inline
from xgboost.sklearn import XGBClassifier 

In [3]:
train = pd.read_csv("../input/videogame/train.csv",index_col="id")
test = pd.read_csv("../input/testvideogame/test.csv",index_col="id")

In [4]:
#display all data
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [5]:
ntrain=train.shape[0]
ntest=test.shape[0]
ytrain=train['playtime_forever']
all_data=pd.concat((train,test))

In [6]:
# #check missing values
# all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
# all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
# missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
# missing_data.head()

In [7]:
#fill missing values
all_data['total_positive_reviews'] = all_data['total_positive_reviews'].fillna(all_data['total_positive_reviews'].mode()[0])
all_data['total_negative_reviews'] = all_data['total_negative_reviews'].fillna(all_data['total_negative_reviews'].mode()[0])
all_data['purchase_date'] = all_data['purchase_date'].fillna(all_data['purchase_date'].mode()[0])

In [8]:
#feature engineering
#deal with dates
all_data['purchase_date']= pd.to_datetime(all_data['purchase_date'])
all_data['release_date']= pd.to_datetime(all_data['release_date'])
all_data['purchase_year']=all_data['purchase_date'].dt.year
all_data['purchase_month']=all_data['purchase_date'].dt.month
all_data['purchase_day']=all_data['purchase_date'].dt.day
all_data['purchase_week']=all_data['purchase_date'].dt.dayofweek
all_data['release_year']=all_data['release_date'].dt.year
all_data['release_month']=all_data['release_date'].dt.month
all_data['release_day']=all_data['release_date'].dt.day
all_data['release_week']=all_data['release_date'].dt.dayofweek
all_data['datedistance']=(all_data['purchase_date']-all_data['release_date']).dt.days

In [9]:
#generate new feature
all_data['hot']=all_data['total_positive_reviews']+all_data['total_negative_reviews']
all_data['distance']=(2019-all_data['release_year'])+all_data['release_month']/12
all_data['averhot']=all_data['hot']/all_data['distance']

In [10]:
#one-hot encoding
all_data['is_free']=all_data['is_free'].apply(lambda x : 1 if x==True else 0)
dumgenres =all_data['genres'].str.get_dummies(",").add_prefix("genres_")
dumcategories=all_data['categories'].str.get_dummies(",").add_prefix("categories_")
dumtags=all_data['tags'].str.get_dummies(",").add_prefix("tags_")
all_data=pd.concat([all_data,dumgenres],axis=1)
all_data=pd.concat([all_data,dumcategories],axis=1)
all_data=pd.concat([all_data,dumtags],axis=1)
all_data=all_data.drop(['genres','categories','tags'],axis=1)

In [11]:
#target encoding
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["median", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.median()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["median"] * smoothing
    averages.drop(["median", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_median').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_median').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [12]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [13]:
# Target encode purchase_year
trainpurchase_year,testpurchase_year=target_encode(train["purchase_year"],test["purchase_year"],target=train['playtime_forever'],min_samples_leaf=10,smoothing=10,noise_level=0.01)
# Target encode purchase_month
trainpurchase_month,testpurchase_month=target_encode(train["purchase_month"],test["purchase_month"],target=train['playtime_forever'],min_samples_leaf=10,smoothing=10,noise_level=0.01)
# Target encode purchase_day
trainpurchase_day,testpurchase_day=target_encode(train["purchase_day"],test["purchase_day"],target=train['playtime_forever'],min_samples_leaf=10,smoothing=10,noise_level=0.01)
# Target encode purchase_week
trainpurchase_week,testpurchase_week=target_encode(train["purchase_week"],test["purchase_week"],target=train['playtime_forever'],min_samples_leaf=10,smoothing=10,noise_level=0.01)
# Target encode release_year
trainrelease_year,testrelease_year=target_encode(train["release_year"],test["release_year"],target=train['playtime_forever'],min_samples_leaf=10,smoothing=10,noise_level=0.01)
# Target encode release_month
trainrelease_month,testrelease_month=target_encode(train["release_month"],test["release_month"],target=train['playtime_forever'],min_samples_leaf=10,smoothing=10,noise_level=0.01)
# Target encode release_day
trainrelease_day,testrelease_day= target_encode(train["release_day"],test["release_day"],target=train['playtime_forever'],min_samples_leaf=10,smoothing=10,noise_level=0.01) 
# Target encode release_week
trainrelease_week,testrelease_week= target_encode(train["release_week"],test["release_week"],target=train['playtime_forever'],min_samples_leaf=10,smoothing=10,noise_level=0.01)                         
train = pd.concat([train,trainpurchase_year,trainpurchase_month,trainpurchase_day,trainpurchase_week,trainrelease_year,trainrelease_month,trainrelease_day,trainrelease_week],axis =1)
test = pd.concat([test,testpurchase_year,testpurchase_month,testpurchase_day,testpurchase_week,testrelease_year,testrelease_month,testrelease_day,testrelease_week],axis =1)

In [14]:
corrmat =train.corr()
bubu=corrmat[(corrmat['playtime_forever']>0.2)|(corrmat['playtime_forever']<-0.2)]
bubu.head(200)

,is_free,playtime_forever,price,total_negative_reviews,total_positive_reviews,purchase_year,purchase_month,purchase_day,purchase_week,release_year,release_month,release_day,release_week,datedistance,hot,distance,averhot,genres_Action,genres_Adventure,genres_Animation & Modeling,genres_Audio Production,genres_Casual,genres_Design & Illustration,genres_Early Access,genres_Free to Play,genres_Gore,genres_Indie,genres_Massively Multiplayer,genres_Nudity,genres_RPG,genres_Racing,genres_Sexual Content,genres_Simulation,genres_Sports,genres_Strategy,genres_Utilities,genres_Violent,categories_Captions available,categories_Co-op,categories_Commentary available,categories_Cross-Platform Multiplayer,categories_Full controller support,categories_In-App Purchases,categories_Includes Source SDK,categories_Includes level editor,categories_Local Co-op,categories_Local Multi-Player,categories_MMO,categories_Multi-player,categories_Online Co-op,categories_Online Multi-Player,categories_Partial Controller Support,categories_Remote Play on Phone,categories_Remote Play on TV,categories_Remote Play on Tablet,categories_Shared/Split Screen,categories_Single-player,categories_Stats,categories_Steam Achievements,categories_Steam Cloud,categories_Steam Leaderboards,categories_Steam Trading Cards,categories_Steam Workshop,categories_SteamVR Collectibles,categories_VR Support,categories_Valve Anti-Cheat enabled,tags_1980s,tags_1990's,tags_2.5D,tags_2D,tags_3D,tags_3D Platformer,tags_3D Vision,tags_4 Player Local,tags_4X,tags_ATV,tags_Action,tags_Action RPG,tags_Action-Adventure,tags_Addictive,tags_Adventure,tags_Aliens,tags_Alternate History,tags_America,tags_Animation & Modeling,tags_Anime,tags_Arcade,tags_Arena Shooter,tags_Artificial Intelligence,tags_Assassin,tags_Atmospheric,tags_Audio Production,tags_Automation,tags_Base Building,tags_Based On A Novel,tags_Batman,tags_Battle Royale,tags_Beat 'em up,tags_Beautiful,tags_Benchmark,tags_Bikes,tags_Blood,tags_Board Game,tags_Building,tags_Bullet Hell,tags_Bullet Time,tags_CRPG,tags_Capitalism,tags_Card Game,tags_Cartoon,tags_Cartoony,tags_Casual,tags_Cats,tags_Character Action Game,tags_Character Customization,tags_Chess,tags_Choices Matter,tags_Choose Your Own Adventure,tags_Cinematic,tags_City Builder,tags_Classic,tags_Clicker,tags_Co-op,tags_Co-op Campaign,tags_Cold War,tags_Colorful,tags_Comedy,tags_Comic Book,tags_Competitive,tags_Conspiracy,tags_Controller,tags_Crafting,tags_Crime,tags_Crowdfunded,tags_Cult Classic,tags_Cute,tags_Cyberpunk,tags_Dark,tags_Dark Comedy,tags_Dark Fantasy,tags_Dark Humor,tags_Dating Sim,tags_Demons,tags_Design & Illustration,tags_Destruction,tags_Detective,tags_Difficult,tags_Dinosaurs,tags_Diplomacy,tags_Documentary,tags_Dragons,tags_Drama,tags_Driving,tags_Dungeon Crawler,tags_Dungeons & Dragons,tags_Dynamic Narration,tags_Dystopian,tags_Early Access,tags_Economy,tags_Education,tags_Emotional,tags_Epic,tags_Episodic,tags_Experience,tags_Experimental,tags_Exploration,tags_FMV,tags_FPS,tags_Family Friendly,tags_Fantasy,tags_Fast-Paced,tags_Female Protagonist,tags_Fighting,tags_First-Person,tags_Flight,tags_Free to Play,tags_Funny,tags_Futuristic,tags_Game Development,tags_GameMaker,tags_Games Workshop,tags_God Game,tags_Gore,tags_Gothic,tags_Grand Strategy,tags_Great Soundtrack,tags_Gun Customization,tags_Hack and Slash,tags_Hacking,tags_Hand-drawn,tags_Heist,tags_Hex Grid,tags_Hidden Object,tags_Historical,tags_Horror,tags_Horses,tags_Hunting,tags_Illuminati,tags_Immersive Sim,tags_Indie,tags_Intentionally Awkward Controls,tags_Interactive Fiction,tags_Inventory Management,tags_Investigation,tags_Isometric,tags_JRPG,tags_Kickstarter,tags_LGBTQ+,tags_Lara Croft,tags_Level Editor,tags_Linear,tags_Local Co-Op,tags_Local Multiplayer,tags_Logic,tags_Loot,tags_Lore-Rich,tags_Lovecraftian,tags_MMORPG,tags_MOBA,tags_Magic,tags_Management,tags_Mars,tags_Martial Arts,tags_Massively Multiplayer,tags_Masterpiece,tags_Mature,tags_Mechs,tags_Medieval,tags_Memes,tags_Metroidvania,ta

In [15]:
train=train.ix[:,['averhot','tags_Co-op','tags_Crafting','tags_PvP','purchase_week_median']]
test=test.ix[:,['averhot','tags_Co-op','tags_Crafting','tags_PvP','purchase_week_median']]

In [16]:
#scaling the data
def scaling_data(df):
    min_max_scaler = MinMaxScaler()
    df['averhot'] = min_max_scaler.fit_transform(np.array(df['averhot']).reshape(-1,1))
    df['tags_Co-op'] = min_max_scaler.fit_transform(np.array(df['tags_Co-op']).reshape(-1,1))
    df['tags_Crafting'] = min_max_scaler.fit_transform(np.array(df['tags_Crafting']).reshape(-1,1))
    df['tags_PvP'] = min_max_scaler.fit_transform(np.array(df['tags_PvP']).reshape(-1,1))
    df['purchase_week_median'] = min_max_scaler.fit_transform(np.array(df['purchase_week_median']).reshape(-1,1))
    return df

In [17]:
train = scaling_data(train)
test=scaling_data(test)

In [18]:
# from sklearn.linear_model import RidgeCV
# ridgecv = RidgeCV(alphas=[0.01, 0.1, 0.5, 1, 5, 7, 10, 30,100, 200])
# ridgecv.fit(train, ytrain)
# ridgecv.alpha_ 

In [19]:
# lasso = Lasso(alpha=0.01)  
# lasso.fit(train,ytrain)
# test['playtime_forever'] = lasso.predict(test)
# test['playtime_forever']=test['playtime_forever'].apply(lambda x:0 if x<0 else x)
# result=test.loc[:,['playtime_forever']]
# result.to_csv("submission.csv")

In [20]:
# KNeighbors Regression
knn = KNeighborsRegressor()
knn.fit(train,ytrain)
test['playtime_forever'] = knn.predict(test)
test['playtime_forever']=test['playtime_forever'].apply(lambda x:0 if x<0 else x)
result=test.loc[:,['playtime_forever']]
result.to_csv("submission.csv")